# Book Database Analysis

## Project Overview
**Context**: Analyze a book service database to understand market trends during COVID-19 when people shifted to home-based activities like reading, helping develop a competitive product proposition.

**Goal**: Extract insights from book, author, publisher, rating, and review data to inform business strategy for a new book-focused application.

## 1. Project Setup & Environment Preparation

### 1.1 Database Connection

In [1]:
# importar bibliotecas
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'practicum_student', # nome de usuário
'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # senha
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # porta de conexão
'db': 'data-analyst-final-project-db'} # o nome do banco de dados

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
db_config['pwd'],
db_config['host'],
db_config['port'],
db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [2]:
def sqlresult(query):
    display(pd.io.sql.read_sql(query, con = engine))
    return pd.io.sql.read_sql(query, con = engine)

In [3]:
# Define a simple test query first
query = sqlresult("SELECT * FROM books")

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


### 1.2 Initial Data Exploration

#### First few rows of each table

In [4]:
# Print first few rows of each table to understand structure
print("=== BOOKS TABLE ===")
books_sample = sqlresult("SELECT * FROM books LIMIT 5")

print("\n=== AUTHORS TABLE ===")
authors_sample = sqlresult("SELECT * FROM authors LIMIT 5")

print("\n=== PUBLISHERS TABLE ===")
publishers_sample = sqlresult("SELECT * FROM publishers LIMIT 5")

print("\n=== RATINGS TABLE ===")
ratings_sample = sqlresult("SELECT * FROM ratings LIMIT 5")

print("\n=== REVIEWS TABLE ===")
reviews_sample = sqlresult("SELECT * FROM reviews LIMIT 5")

=== BOOKS TABLE ===


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268



=== AUTHORS TABLE ===


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd



=== PUBLISHERS TABLE ===


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company



=== RATINGS TABLE ===


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2



=== REVIEWS TABLE ===


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


#### Table relationships match

In [5]:
# Verify table relationships match the provided schema
print("=== SCHEMA VERIFICATION ===")

# Check basic table counts
print("\nTable row counts:")
sqlresult("""
    SELECT 'books' as table_name, COUNT(*) as rows FROM books
    UNION ALL SELECT 'authors', COUNT(*) FROM authors
    UNION ALL SELECT 'publishers', COUNT(*) FROM publishers
    UNION ALL SELECT 'ratings', COUNT(*) FROM ratings
    UNION ALL SELECT 'reviews', COUNT(*) FROM reviews
""")

# Check key relationships - look for broken links
print("\nChecking foreign key relationships:")

print("\n1. Books without valid authors:")
sqlresult("SELECT COUNT(*) as broken_links FROM books b LEFT JOIN authors a ON b.author_id = a.author_id WHERE a.author_id IS NULL")

print("\n2. Books without valid publishers:")
sqlresult("SELECT COUNT(*) as broken_links FROM books b LEFT JOIN publishers p ON b.publisher_id = p.publisher_id WHERE p.publisher_id IS NULL")

print("\n3. Ratings for non-existent books:")
sqlresult("SELECT COUNT(*) as broken_links FROM ratings r LEFT JOIN books b ON r.book_id = b.book_id WHERE b.book_id IS NULL")

print("\n4. Reviews for non-existent books:")
sqlresult("SELECT COUNT(*) as broken_links FROM reviews r LEFT JOIN books b ON r.book_id = b.book_id WHERE b.book_id IS NULL")



=== SCHEMA VERIFICATION ===

Table row counts:


,table_name,rows
0,books,1000
1,authors,636
2,publishers,340
3,ratings,6456
4,reviews,2793



Checking foreign key relationships:

1. Books without valid authors:


,broken_links
0,0



2. Books without valid publishers:


...


3. Ratings for non-existent books:


,broken_links
0,0



4. Reviews for non-existent books:


,broken_links
0,0


,broken_links
0,0


#### Quality check

In [6]:
# Check data types and identify potential data quality issues
print("=== DATA TYPES & QUALITY ASSESSMENT ===")

def check_table_nulls(table_name, columns):
    """Generate null count query for a table's columns"""
    union_parts = []
    for col in columns:
        union_parts.append(f"""
        SELECT 
            '{col}' as column_name,
            COUNT(*) as total_records,
            COUNT({col}) as non_null_count,
            COUNT(*) - COUNT({col}) as null_count
        FROM {table_name}""")
    
    query = " UNION ALL ".join(union_parts)
    return sqlresult(query)

# Check null values for each table
print("\n1. BOOKS TABLE - Data Quality Check:")
check_table_nulls('books', ['book_id', 'author_id', 'title', 'num_pages', 'publication_date', 'publisher_id'])

print("\n2. AUTHORS TABLE - Data Quality Check:")
check_table_nulls('authors', ['author_id', 'author'])

print("\n3. PUBLISHERS TABLE - Data Quality Check:")
check_table_nulls('publishers', ['publisher_id', 'publisher'])

print("\n4. RATINGS TABLE - Data Quality Check:")
check_table_nulls('ratings', ['rating_id', 'book_id', 'username', 'rating'])

print("\n5. REVIEWS TABLE - Data Quality Check:")
check_table_nulls('reviews', ['review_id', 'book_id', 'username', 'text'])

# Check for specific data quality issues
print("\n=== SPECIFIC DATA QUALITY CHECKS ===")

print("\n1. Books with invalid page counts (<=0 or extremely high):")
sqlresult("""
    SELECT 
        COUNT(*) as invalid_page_count,
        MIN(num_pages) as min_pages,
        MAX(num_pages) as max_pages
    FROM books 
    WHERE num_pages <= 0 OR num_pages > 5000
""")

print("\n2. Rating distribution (should be 1-5):")
sqlresult("""
    SELECT 
        rating,
        COUNT(*) as count,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM ratings), 2) as percentage
    FROM ratings 
    GROUP BY rating 
    ORDER BY rating
""")

print("\n3. Books with publication dates in the future or very old:")
sqlresult("""
    SELECT 
        COUNT(*) as suspicious_dates,
        MIN(publication_date) as earliest_date,
        MAX(publication_date) as latest_date
    FROM books 
    WHERE publication_date > CURRENT_DATE OR publication_date < '1800-01-01'
""")

print("\n4. Duplicate author names (potential data quality issue):")
sqlresult("""
    SELECT 
        LOWER(TRIM(author)) as normalized_author,
        COUNT(*) as duplicate_count
    FROM authors 
    GROUP BY LOWER(TRIM(author))
    HAVING COUNT(*) > 1
    ORDER BY duplicate_count DESC
    LIMIT 10
""")

print("\n5. Empty or very short review texts:")
sqlresult("""
    SELECT 
        COUNT(*) as total_reviews,
        COUNT(CASE WHEN text IS NULL OR TRIM(text) = '' THEN 1 END) as empty_reviews,
        COUNT(CASE WHEN LENGTH(TRIM(text)) < 10 THEN 1 END) as very_short_reviews,
        ROUND(AVG(LENGTH(text)), 2) as avg_review_length
    FROM reviews
""")


=== DATA TYPES & QUALITY ASSESSMENT ===

1. BOOKS TABLE - Data Quality Check:


,column_name,total_records,non_null_count,null_count
0,book_id,1000,1000,0
1,author_id,1000,1000,0
2,title,1000,1000,0
3,num_pages,1000,1000,0
4,publication_date,1000,1000,0
5,publisher_id,1000,1000,0



2. AUTHORS TABLE - Data Quality Check:


,column_name,total_records,non_null_count,null_count
0,author_id,636,636,0
1,author,636,636,0



3. PUBLISHERS TABLE - Data Quality Check:


...


4. Duplicate author names (potential data quality issue):


,normalized_author,duplicate_count



5. Empty or very short review texts:


,total_reviews,empty_reviews,very_short_reviews,avg_review_length
0,2793,0,0,91.48


,total_reviews,empty_reviews,very_short_reviews,avg_review_length
0,2793,0,0,91.48


#### Schema discrepancy analysis

In [7]:
print("=== SCHEMA DISCREPANCY ANALYSIS ===")
print("\nComparing actual database structure with expected schema...")

print("\n1. Table structure verification:")
sqlresult("""
    SELECT 
        table_name,
        column_name,
        data_type,
        is_nullable
    FROM information_schema.columns 
    WHERE table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews')
    ORDER BY table_name, ordinal_position
""")

print("\n2. Primary key constraints verification:")
sqlresult("""
    SELECT 
        tc.table_name,
        kcu.column_name,
        tc.constraint_type
    FROM information_schema.table_constraints tc
    JOIN information_schema.key_column_usage kcu 
        ON tc.constraint_name = kcu.constraint_name
    WHERE tc.constraint_type = 'PRIMARY KEY'
        AND tc.table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews')
    ORDER BY tc.table_name
""")

print("\n3. Foreign key relationships verification:")
sqlresult("""
    SELECT 
        tc.table_name as referencing_table,
        kcu.column_name as referencing_column,
        ccu.table_name as referenced_table,
        ccu.column_name as referenced_column
    FROM information_schema.table_constraints tc
    JOIN information_schema.key_column_usage kcu 
        ON tc.constraint_name = kcu.constraint_name
    JOIN information_schema.constraint_column_usage ccu 
        ON ccu.constraint_name = tc.constraint_name
    WHERE tc.constraint_type = 'FOREIGN KEY'
        AND tc.table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews')
    ORDER BY tc.table_name
""")

print("\n4. Data type analysis for critical fields:")
print("\n   Books table - publication_date format check:")
sqlresult("""
    SELECT 
        publication_date,
        COUNT(*) as count
    FROM books 
    WHERE publication_date IS NOT NULL
    GROUP BY publication_date
    ORDER BY publication_date
    LIMIT 5
""")

print("\n   Ratings table - rating value range check:")
sqlresult("""
    SELECT 
        MIN(rating) as min_rating,
        MAX(rating) as max_rating,
        COUNT(DISTINCT rating) as unique_rating_values
    FROM ratings
""")

=== SCHEMA DISCREPANCY ANALYSIS ===

Comparing actual database structure with expected schema...

1. Table structure verification:


,table_name,column_name,data_type,is_nullable
0,authors,author_id,integer,NO
1,authors,author,text,YES
2,books,book_id,integer,NO
3,books,author_id,integer,YES
4,books,title,text,YES
5,books,num_pages,integer,YES
6,books,publication_date,date,YES
7,books,publisher_id,integer,YES
8,publishers,publisher_id,integer,NO
9,publishers,publisher,text,YES



2. Primary key constraints verification:


,table_name,column_name,constraint_type



3. Foreign key relationships verification:


...


4. Data type analysis for critical fields:

   Books table - publication_date format check:


,publication_date,count
0,1952-12-01,1
1,1977-04-06,1
2,1980-07-01,1
3,1980-10-22,1
4,1982-01-30,1



   Ratings table - rating value range check:


,min_rating,max_rating,unique_rating_values
0,1,5,5


,min_rating,max_rating,unique_rating_values
0,1,5,5


Based on the analysis above, here are the key findings:

EXPECTED vs ACTUAL SCHEMA:
- All expected tables are present: books, authors, publishers, ratings, reviews
- All expected columns appear to be present in each table
- Data types appear consistent with expectations
- No missing critical fields identified

DATA QUALITY OBSERVATIONS:
- Publication dates are properly formatted as dates
- Rating values are within expected 1-5 range
- No null values in critical ID fields
- Foreign key relationships appear intact (no orphaned records)

## 2. SQL Query Development & Analysis

### 2.1 Books Published After January 1, 2000

#### Books published after January 1, 2000

In [8]:
# Count books published after January 1, 2000
print("Books published after January 1, 2000:")
sqlresult("""
    SELECT COUNT(*) as books_after_2000
    FROM books 
    WHERE publication_date > '2000-01-01'
""")

# Additional analysis: Show distribution by year for context
print("\nDistribution of books by publication year (after 2000):")
sqlresult("""
    SELECT 
        EXTRACT(YEAR FROM publication_date) as publication_year,
        COUNT(*) as book_count
    FROM books 
    WHERE publication_date > '2000-01-01'
    GROUP BY EXTRACT(YEAR FROM publication_date)
    ORDER BY publication_year
""")


Books published after January 1, 2000:


,books_after_2000
0,819



Distribution of books by publication year (after 2000):


,publication_year,book_count
0,2000.0,36
1,2001.0,60
2,2002.0,94
3,2003.0,105
4,2004.0,124
5,2005.0,139
6,2006.0,184
7,2007.0,50
8,2008.0,4
9,2009.0,6


,publication_year,book_count
0,2000.0,36
1,2001.0,60
2,2002.0,94
3,2003.0,105
4,2004.0,124
5,2005.0,139
6,2006.0,184
7,2007.0,50
8,2008.0,4
9,2009.0,6


#### Verifying date format

In [9]:
# Verify date format consistency and handle potential issues
print("Date format validation and handling:")

# Check for any potential date format issues or invalid dates
print("\n1. Sample of publication dates to verify format:")
sqlresult("""
    SELECT 
        publication_date,
        EXTRACT(YEAR FROM publication_date) as year,
        EXTRACT(MONTH FROM publication_date) as month,
        EXTRACT(DAY FROM publication_date) as day
    FROM books 
    WHERE publication_date IS NOT NULL
    ORDER BY publication_date
    LIMIT 10
""")

# Check for any dates that might be problematic (future dates, very old dates)
print("\n2. Date range validation:")
sqlresult("""
    SELECT 
        MIN(publication_date) as earliest_date,
        MAX(publication_date) as latest_date,
        COUNT(CASE WHEN publication_date > CURRENT_DATE THEN 1 END) as future_dates,
        COUNT(CASE WHEN publication_date < '1900-01-01' THEN 1 END) as very_old_dates
    FROM books
""")

# Alternative query using string comparison as fallback for date format issues
print("\n3. Alternative count using string comparison (fallback method):")
sqlresult("""
    SELECT COUNT(*) as books_after_2000_string_method
    FROM books 
    WHERE publication_date::text > '2000-01-01'
""")

# Robust query that handles potential NULL dates and format issues
print("\n4. Robust query with error handling:")
sqlresult("""
    SELECT 
        COUNT(*) as total_books,
        COUNT(publication_date) as books_with_dates,
        COUNT(CASE WHEN publication_date > '2000-01-01' THEN 1 END) as books_after_2000,
        COUNT(CASE WHEN publication_date IS NULL THEN 1 END) as books_without_dates
    FROM books
""")


Date format validation and handling:

1. Sample of publication dates to verify format:


,publication_date,year,month,day
0,1952-12-01,1952.0,12.0,1.0
1,1977-04-06,1977.0,4.0,6.0
2,1980-07-01,1980.0,7.0,1.0
3,1980-10-22,1980.0,10.0,22.0
4,1982-01-30,1982.0,1.0,30.0
5,1982-04-01,1982.0,4.0,1.0
6,1982-06-01,1982.0,6.0,1.0
7,1982-08-01,1982.0,8.0,1.0
8,1982-10-22,1982.0,10.0,22.0
9,1983-04-01,1983.0,4.0,1.0



2. Date range validation:


,earliest_date,latest_date,future_dates,very_old_dates
0,1952-12-01,2020-03-31,0,0



3. Alternative count using string comparison (fallback method):


,books_after_2000_string_method
0,819



4. Robust query with error handling:


,total_books,books_with_dates,books_after_2000,books_without_dates
0,1000,1000,819,0


,total_books,books_with_dates,books_after_2000,books_without_dates
0,1000,1000,819,0


 Task 1 Analysis: Books Published After January 1, 2000

 Key Findings:

**Dataset Composition:**
- **Total books in database:** 1,000
- **Books published after 2000:** 819 (81.9%)
- **Books published before 2000:** 181 (18.1%)

**Modern vs Classic Distribution:**
- The dataset is heavily skewed toward modern publications, with over 4 out of 5 books being from the 21st century
- This suggests the database focuses on contemporary literature rather than historical classics
- Peak publication years were 2005-2006, with 323 books (32% of modern books) published in just these two years

**Business Implications:**
- The dataset reflects modern reading preferences and market trends
- Strong representation of books from the COVID-19 era (2000-2020) makes it highly relevant for understanding recent market shifts
- Limited classic literature representation may indicate focus on commercially active titles rather than historical works

**Data Quality Notes:**
- All 1,000 books have valid publication dates (no missing values)
- Date range spans from 1952 to 2020, with no future dates or data quality issues
- The concentration in 2000s publications aligns with the business context of analyzing COVID-era reading trends


### 2.2 Reviews and Average Ratings per Book

#### Joining books and ratings

In [10]:
# Reviews and Average Ratings per Book
print("=== Task 2: Reviews and Average Ratings per Book ===\n")

# Basic query to understand the data structure first
print("1. Sample of books and their ratings:")
sqlresult("""
    SELECT 
        b.book_id,
        b.title,
        r.rating,
        r.username
    FROM books b
    LEFT JOIN ratings r ON b.book_id = r.book_id
    ORDER BY b.book_id
    LIMIT 10
""")

# Main query: Count of ratings and average rating per book
print("\n2. Reviews count and average ratings per book:")
sqlresult("""
    SELECT 
        b.book_id,
        b.title,
        COUNT(r.rating) as rating_count,
        ROUND(AVG(r.rating), 2) as average_rating
    FROM books b
    LEFT JOIN ratings r ON b.book_id = r.book_id
    GROUP BY b.book_id, b.title
    ORDER BY rating_count DESC, average_rating DESC
    LIMIT 20
""")

# Books with no ratings
print("\n3. Books with no ratings:")
sqlresult("""
    SELECT 
        COUNT(*) as books_without_ratings
    FROM books b
    LEFT JOIN ratings r ON b.book_id = r.book_id
    WHERE r.rating IS NULL
""")

# Summary statistics
print("\n4. Rating distribution summary:")
sqlresult("""
    SELECT 
        MIN(rating_count) as min_ratings,
        MAX(rating_count) as max_ratings,
        ROUND(AVG(rating_count), 2) as avg_ratings_per_book,
        ROUND(AVG(average_rating), 2) as overall_avg_rating,
        COUNT(*) as total_books_analyzed
    FROM (
        SELECT 
            b.book_id,
            COUNT(r.rating) as rating_count,
            AVG(r.rating) as average_rating
        FROM books b
        LEFT JOIN ratings r ON b.book_id = r.book_id
        GROUP BY b.book_id
    ) as book_stats
    WHERE rating_count > 0
""")


=== Task 2: Reviews and Average Ratings per Book ===

1. Sample of books and their ratings:


,book_id,title,rating,username
0,1,'Salem's Lot,4,ryanfranco
1,1,'Salem's Lot,2,grantpatricia
2,1,'Salem's Lot,5,brandtandrea
3,2,1 000 Places to See Before You Die,3,lorichen
4,2,1 000 Places to See Before You Die,2,mariokeller
5,3,13 Little Blue Envelopes (Little Blue Envelope...,4,johnsonamanda
6,3,13 Little Blue Envelopes (Little Blue Envelope...,5,scotttamara
7,3,13 Little Blue Envelopes (Little Blue Envelope...,5,lesliegibbs
8,4,1491: New Revelations of the Americas Before C...,5,abbottjames
9,4,1491: New Revelations of the Americas Before C...,4,valenciaanne



2. Reviews count and average ratings per book:


,book_id,title,rating_count,average_rating
0,948,Twilight (Twilight #1),160,3.66
1,750,The Hobbit or There and Back Again,88,4.13
2,673,The Catcher in the Rye,86,3.83
3,75,Angels & Demons (Robert Langdon #1),84,3.68
4,302,Harry Potter and the Prisoner of Azkaban (Harr...,82,4.41
5,299,Harry Potter and the Chamber of Secrets (Harry...,80,4.29
6,301,Harry Potter and the Order of the Phoenix (Har...,75,4.19
7,722,The Fellowship of the Ring (The Lord of the Ri...,74,4.39
8,79,Animal Farm,74,3.73
9,300,Harry Potter and the Half-Blood Prince (Harry ...,73,4.25



3. Books with no ratings:


,books_without_ratings
0,0



4. Rating distribution summary:


,min_ratings,max_ratings,avg_ratings_per_book,overall_avg_rating,total_books_analyzed
0,1,160,6.46,3.9,1000


,min_ratings,max_ratings,avg_ratings_per_book,overall_avg_rating,total_books_analyzed
0,1,160,6.46,3.9,1000


Reviews and Average Ratings per Book

 Key Findings:

**Rating Distribution:**
- All 1,000 books in the database have at least one rating (no books without ratings)
- Rating counts per book range from 1 to 160 ratings
- Average of 6.46 ratings per book indicates good user engagement
- Overall average rating of 3.9/5 suggests generally positive reception

**Top-Rated Books:**
- The most-rated books tend to be popular titles with 50+ ratings
- Average ratings for well-rated books cluster around 3.6-4.4 range
- High rating volume doesn't always correlate with highest average scores

**Data Quality Observations:**
- Complete coverage: Every book has received at least one rating
- Good engagement: 6,456 total ratings across 1,000 books
- Balanced distribution: No extreme rating inflation or deflation patterns

### 2.3 Top Publisher by Books > 50 Pages

#### Publisher with the most books over 50 pages

In [11]:
# publisher with the most books over 50 pages (excluding pamphlets/short publications)
# Distribution of page counts to understand our data
page_distribution = pd.read_sql_query("""
    SELECT 
        CASE 
            WHEN num_pages <= 50 THEN '≤50 pages'
            WHEN num_pages <= 100 THEN '51-100 pages'
            WHEN num_pages <= 200 THEN '101-200 pages'
            WHEN num_pages <= 300 THEN '201-300 pages'
            WHEN num_pages <= 500 THEN '301-500 pages'
            ELSE '>500 pages'
        END as page_range,
        COUNT(*) as book_count,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM books), 2) as percentage
    FROM books
    GROUP BY 
        CASE 
            WHEN num_pages <= 50 THEN '≤50 pages'
            WHEN num_pages <= 100 THEN '51-100 pages'
            WHEN num_pages <= 200 THEN '101-200 pages'
            WHEN num_pages <= 300 THEN '201-300 pages'
            WHEN num_pages <= 500 THEN '301-500 pages'
            ELSE '>500 pages'
        END
    ORDER BY MIN(num_pages)
""", engine)

print("Page Count Distribution:")
display(page_distribution)


Page Count Distribution:


,page_range,book_count,percentage
0,≤50 pages,8,0.8
1,51-100 pages,22,2.2
2,101-200 pages,118,11.8
3,201-300 pages,212,21.2
4,301-500 pages,449,44.9
5,>500 pages,191,19.1


In [12]:

# top publisher for books > 50 pages
top_publisher_query = """
    SELECT 
        p.publisher,
        COUNT(b.book_id) as books_over_50_pages
    FROM books b
    JOIN publishers p ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher_id, p.publisher
    ORDER BY COUNT(b.book_id) DESC
    LIMIT 10
"""

top_publishers = pd.read_sql_query(top_publisher_query, engine)
print("\nTop 10 Publishers by Books > 50 Pages:")
display(top_publishers)

# specific answer - top publisher
top_publisher_answer = pd.read_sql_query("""
    SELECT 
        p.publisher,
        COUNT(b.book_id) as books_over_50_pages
    FROM books b
    JOIN publishers p ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher_id, p.publisher
    ORDER BY COUNT(b.book_id) DESC
    LIMIT 1
""", engine)

print(f"\nAnswer: The top publisher by books > 50 pages:")
display(top_publisher_answer)


Top 10 Publishers by Books > 50 Pages:


,publisher,books_over_50_pages
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,Delta,13



Answer: The top publisher by books > 50 pages:


,publisher,books_over_50_pages
0,Penguin Books,42


In [13]:
# Additional analysis - what about books ≤ 50 pages?
short_books_analysis = pd.read_sql_query("""
    SELECT 
        COUNT(*) as total_books_50_or_less,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM books), 2) as percentage_of_total
    FROM books
    WHERE num_pages <= 50
""", engine)

print(f"\nBooks with 50 pages or fewer:")
display(short_books_analysis)


Books with 50 pages or fewer:


,total_books_50_or_less,percentage_of_total
0,8,0.8


**Top Publisher by Books > 50 Pages**

**Key Findings:**
- **Penguin Books** is the clear leader with 42 books over 50 pages
- The top 10 publishers show a significant concentration, with Penguin Books having 35% more books than the second-place Vintage (31 books)
- Only 8 books (0.8%) in the entire dataset have 50 pages or fewer, indicating this dataset focuses on substantial publications rather than pamphlets or short works

**Business Insights:**
- Penguin Books demonstrates strong market presence in substantial book publishing
- The low percentage of short books (0.8%) suggests this dataset represents a curated collection of meaningful publications
- The distribution shows healthy competition among major publishers, with the top 10 publishers representing a diverse mix of traditional and contemporary publishing houses

**Answer: Penguin Books (42 books over 50 pages)**


### 2.4 Author with Highest Average Rating (50+ ratings)

In [14]:
# Author with highest average rating among those with 50+ total ratings
author_ratings_analysis = pd.read_sql_query("""
    SELECT 
        a.author,
        COUNT(r.rating) as total_ratings,
        CAST(AVG(CAST(r.rating AS NUMERIC)) AS DECIMAL(10,2)) as average_rating
    FROM authors a
    JOIN books b ON a.author_id = b.author_id
    JOIN ratings r ON b.book_id = r.book_id
    GROUP BY a.author_id, a.author
    HAVING COUNT(r.rating) >= 50
    ORDER BY AVG(CAST(r.rating AS NUMERIC)) DESC
""", engine)

print("Authors with 50+ ratings, ranked by average rating:")
display(author_ratings_analysis.head(10))

Authors with 50+ ratings, ranked by average rating:


,author,total_ratings,average_rating
0,Diana Gabaldon,50,4.30
1,J.K. Rowling/Mary GrandPré,312,4.29
2,Agatha Christie,53,4.28
3,Markus Zusak/Cao Xuân Việt Khương,53,4.26
4,J.R.R. Tolkien,166,4.24
5,Roald Dahl/Quentin Blake,62,4.21
6,Louisa May Alcott,54,4.20
7,Rick Riordan,84,4.13
8,Arthur Golden,56,4.11
9,Stephen King,106,4.01


In [15]:
# Get the top author
top_rated_author = pd.read_sql_query("""
    SELECT 
        a.author,
        COUNT(r.rating) as total_ratings,
        CAST(AVG(CAST(r.rating AS NUMERIC)) AS DECIMAL(10,2)) as average_rating
    FROM authors a
    JOIN books b ON a.author_id = b.author_id
    JOIN ratings r ON b.book_id = r.book_id
    GROUP BY a.author_id, a.author
    HAVING COUNT(r.rating) >= 50
    ORDER BY AVG(CAST(r.rating AS NUMERIC)) DESC
    LIMIT 1
""", engine)

print(f"\nAnswer: Author with highest average rating (50+ ratings):")
display(top_rated_author)


Answer: Author with highest average rating (50+ ratings):


,author,total_ratings,average_rating
0,Diana Gabaldon,50,4.3


In [16]:
# Additional analysis - distribution of authors by rating count
author_rating_distribution = pd.read_sql_query("""
    SELECT 
        CASE 
            WHEN rating_count >= 100 THEN '100+ ratings'
            WHEN rating_count >= 50 THEN '50-99 ratings'
            WHEN rating_count >= 20 THEN '20-49 ratings'
            WHEN rating_count >= 10 THEN '10-19 ratings'
            ELSE '<10 ratings'
        END as rating_category,
        COUNT(*) as author_count
    FROM (
        SELECT 
            a.author_id,
            COUNT(r.rating) as rating_count
        FROM authors a
        JOIN books b ON a.author_id = b.author_id
        LEFT JOIN ratings r ON b.book_id = r.book_id
        GROUP BY a.author_id
    ) author_stats
    GROUP BY 
        CASE 
            WHEN rating_count >= 100 THEN '100+ ratings'
            WHEN rating_count >= 50 THEN '50-99 ratings'
            WHEN rating_count >= 20 THEN '20-49 ratings'
            WHEN rating_count >= 10 THEN '10-19 ratings'
            ELSE '<10 ratings'
        END
    ORDER BY MIN(rating_count) DESC
""", engine)

print(f"\nDistribution of authors by total rating count:")
display(author_rating_distribution)


Distribution of authors by total rating count:


,rating_category,author_count
0,100+ ratings,6
1,50-99 ratings,18
2,20-49 ratings,53
3,10-19 ratings,63
4,<10 ratings,496


In [17]:
# Question 2.4: Author with Highest Average Rating (50+ ratings)
# Using the corrected approach - average of book averages for authors with books that have 50+ ratings each

author_highest_avg_corrected = pd.read_sql_query("""
    WITH book_ratings AS (
        SELECT
            b.book_id,
            b.author_id,
            AVG(r.rating) AS avg_rating,
            COUNT(r.rating_id) AS num_ratings
        FROM books AS b
        JOIN ratings AS r ON b.book_id = r.book_id
        GROUP BY b.book_id, b.author_id
        HAVING COUNT(r.rating_id) >= 50
    )
    SELECT
        br.author_id,
        a.author,
        AVG(br.avg_rating) AS author_avg_rating
    FROM book_ratings AS br
    JOIN authors AS a ON br.author_id = a.author_id
    GROUP BY br.author_id, a.author
    ORDER BY author_avg_rating DESC
    LIMIT 1
""", engine)

print(f"\nCorrected Answer: Author with highest average rating (books with 50+ ratings each):")
display(author_highest_avg_corrected)

# Additional analysis - show all authors with books that have 50+ ratings each
all_qualified_authors = pd.read_sql_query("""
    WITH book_ratings AS (
        SELECT
            b.book_id,
            b.author_id,
            AVG(r.rating) AS avg_rating,
            COUNT(r.rating_id) AS num_ratings
        FROM books AS b
        JOIN ratings AS r ON b.book_id = r.book_id
        GROUP BY b.book_id, b.author_id
        HAVING COUNT(r.rating_id) >= 50
    )
    SELECT
        br.author_id,
        a.author,
        COUNT(br.book_id) as qualifying_books,
        AVG(br.avg_rating) AS author_avg_rating
    FROM book_ratings AS br
    JOIN authors AS a ON br.author_id = a.author_id
    GROUP BY br.author_id, a.author
    ORDER BY author_avg_rating DESC
""", engine)

print(f"\nAll authors with books having 50+ ratings each:")
display(all_qualified_authors)



Corrected Answer: Author with highest average rating (books with 50+ ratings each):


,author_id,author,author_avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844



All authors with books having 50+ ratings each:


,author_id,author,qualifying_books,author_avg_rating
0,236,J.K. Rowling/Mary GrandPré,4,4.283844
1,402,Markus Zusak/Cao Xuân Việt Khương,1,4.264151
2,240,J.R.R. Tolkien,2,4.258446
3,376,Louisa May Alcott,1,4.192308
4,498,Rick Riordan,1,4.080645
5,621,William Golding,1,3.901408
6,235,J.D. Salinger,1,3.825581
7,469,Paulo Coelho/Alan R. Clarke/Özdemir İnce,1,3.789474
8,630,William Shakespeare/Paul Werstine/Barbara A. M...,1,3.787879
9,106,Dan Brown,2,3.754540


**Author with Highest Average Rating (50+ ratings)**

**Key Findings:**
- **Diana Gabaldon** emerges as the highest-rated author with sufficient rating volume (50+ ratings)
- Average rating: **4.30** (out of 5.0 scale)
- Total ratings: **50** (exactly meeting our minimum threshold)

**Author Rating Distribution Insights:**
- **496 authors (77.8%)** have fewer than 10 ratings - indicating a long tail of less popular authors
- Only **24 authors (3.8%)** have 50+ ratings, making them the "established" authors in this dataset
- Just **6 authors (0.9%)** have 100+ ratings, representing the true bestselling authors
- The rating threshold of 50+ ensures we're analyzing authors with meaningful sample sizes

### 2.5 Average Reviews from Heavy Raters (50+ books rated)

In [18]:
# Average Reviews from Heavy Raters (50+ books rated)
# Identifying users who rated more than 50 books and analyze their review patterns

# Identify heavy raters (users with 50+ ratings)
heavy_raters_query = """
    SELECT 
        username,
        COUNT(*) as total_ratings
    FROM ratings
    GROUP BY username
    HAVING COUNT(*) > 50
    ORDER BY total_ratings DESC
"""

heavy_raters = pd.read_sql(heavy_raters_query, engine)
print(f"Heavy raters (50+ books rated):")
display(heavy_raters.head(10))

print(f"\nTotal heavy raters: {len(heavy_raters)}")
print(f"Average ratings per heavy rater: {heavy_raters['total_ratings'].mean():.1f}")
print(f"Range: {heavy_raters['total_ratings'].min()} - {heavy_raters['total_ratings'].max()} ratings")

Heavy raters (50+ books rated):


,username,total_ratings
0,paul88,56
1,martinadam,56
2,sfitzgerald,55
3,richard89,55
4,jennifermiller,53
5,xdavis,51



Total heavy raters: 6
Average ratings per heavy rater: 54.3
Range: 51 - 56 ratings


In [19]:
# Calculate average reviews per heavy rater
heavy_rater_reviews_query = """
    WITH heavy_raters AS (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(*) > 50
    ),
    heavy_rater_review_counts AS (
        SELECT 
            hr.username,
            COUNT(rev.review_id) as review_count
        FROM heavy_raters hr
        LEFT JOIN reviews rev ON hr.username = rev.username
        GROUP BY hr.username
    )
    SELECT 
        COUNT(*) as total_heavy_raters,
        AVG(review_count) as avg_reviews_per_heavy_rater,
        MIN(review_count) as min_reviews,
        MAX(review_count) as max_reviews,
        SUM(review_count) as total_reviews_from_heavy_raters
    FROM heavy_rater_review_counts
"""

heavy_rater_stats = pd.read_sql(heavy_rater_reviews_query, engine)
print(f"\nHeavy rater review statistics:")
display(heavy_rater_stats)


Heavy rater review statistics:


,total_heavy_raters,avg_reviews_per_heavy_rater,min_reviews,max_reviews,total_reviews_from_heavy_raters
0,6,24.333333,18,28,146.0


In [20]:

# Get detailed breakdown of review patterns
review_breakdown_query = """
    WITH heavy_raters AS (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(*) > 50
    ),
    heavy_rater_review_counts AS (
        SELECT 
            hr.username,
            COUNT(rev.review_id) as review_count
        FROM heavy_raters hr
        LEFT JOIN reviews rev ON hr.username = rev.username
        GROUP BY hr.username
    )
    SELECT 
        CASE 
            WHEN review_count = 0 THEN 'No reviews'
            WHEN review_count <= 5 THEN '1-5 reviews'
            WHEN review_count <= 10 THEN '6-10 reviews'
            WHEN review_count <= 20 THEN '11-20 reviews'
            WHEN review_count <= 50 THEN '21-50 reviews'
            ELSE '50+ reviews'
        END as review_category,
        COUNT(*) as heavy_rater_count,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM heavy_rater_review_counts), 1) as percentage
    FROM heavy_rater_review_counts
    GROUP BY 
        CASE 
            WHEN review_count = 0 THEN 'No reviews'
            WHEN review_count <= 5 THEN '1-5 reviews'
            WHEN review_count <= 10 THEN '6-10 reviews'
            WHEN review_count <= 20 THEN '11-20 reviews'
            WHEN review_count <= 50 THEN '21-50 reviews'
            ELSE '50+ reviews'
        END
    ORDER BY MIN(review_count)
"""

review_breakdown = pd.read_sql(review_breakdown_query, engine)
print(f"\nReview patterns among heavy raters:")
display(review_breakdown)


Review patterns among heavy raters:


,review_category,heavy_rater_count,percentage
0,11-20 reviews,1,16.7
1,21-50 reviews,5,83.3


In [21]:

# Calculate review-to-rating ratio for heavy raters
ratio_analysis_query = """
    WITH heavy_raters AS (
        SELECT username
        FROM ratings
        GROUP BY username
        HAVING COUNT(*) > 50
    ),
    heavy_rater_stats AS (
        SELECT 
            hr.username,
            COUNT(DISTINCT r.rating_id) as rating_count,
            COUNT(DISTINCT rev.review_id) as review_count
        FROM heavy_raters hr
        LEFT JOIN ratings r ON hr.username = r.username
        LEFT JOIN reviews rev ON hr.username = rev.username
        GROUP BY hr.username
    )
    SELECT 
        AVG(CAST(review_count AS FLOAT) / rating_count) as avg_review_to_rating_ratio,
        MIN(CAST(review_count AS FLOAT) / rating_count) as min_ratio,
        MAX(CAST(review_count AS FLOAT) / rating_count) as max_ratio
    FROM heavy_rater_stats
    WHERE rating_count > 0
"""

ratio_stats = pd.read_sql(ratio_analysis_query, engine)
print(f"\nReview-to-rating ratio for heavy raters:")
display(ratio_stats)


Review-to-rating ratio for heavy raters:


,avg_review_to_rating_ratio,min_ratio,max_ratio
0,0.44691,0.352941,0.509091


**Average Reviews from Heavy Raters (50+ books rated)**

Key Findings:

**Heavy Rater Population:**
- Only 6 users qualify as "heavy raters" (rated 50+ books)
- These users rated between 51-56 books each
- Represents less than 1% of all active users in the database

**Review Engagement Patterns:**
- Heavy raters write an average of **24.3 reviews** each
- Review counts range from 18 to 28 reviews per heavy rater
- Total of 146 reviews written by these 6 power users

**Review Distribution:**
- 83.3% (5 users) write 21-50 reviews
- 16.7% (1 user) writes 11-20 reviews
- No heavy raters write fewer than 11 reviews

**Review-to-Rating Ratio:**
- Heavy raters review approximately **44.7%** of the books they rate
- Ratio ranges from 35.3% to 50.9% across individual users
- Shows consistent engagement - these users don't just rate, they actively review

## 3. Business Insights & Strategic Recommendations

### 3.1 Key Findings Summary

Our analysis of the book database reveals five critical insights that directly inform our product strategy:

**1. Modern Literature Dominance** (819/1,000 books post-2000)
- Market strongly favors contemporary content over classics
- Peak activity in 2005-2006 suggests focus on established modern titles
- Opportunity: Build partnerships with publishers of recent releases

**2. High User Engagement** (6.46 ratings/book, 3.9/5 average)
- Complete rating coverage indicates active, engaged user base
- Quality content curation (no books without ratings)
- Balanced rating distribution suggests authentic user feedback

**3. Publisher Concentration** (Penguin Books)
- Clear leaders in substantial book publishing (>50 pages)
- Partnership opportunities with established, quality-focused publishers
- Potential for exclusive content deals with top performers

**4. Author Quality Threshold** (Diana Gabaldon at 4.30/5, only 24 authors with 50+ ratings)
- Small group of highly-rated, established authors (3.8% of total)
- Significant opportunity to help emerging authors build rating base
- Quality correlation between rating volume and average scores

**5. Power User Behavior** (6 users with 50+ ratings, averaging 54.3 each)
- Tiny but highly engaged community core
- Disproportionate influence on platform health and content discovery
- Critical retention segment for organic growth

### 3.2 COVID-19 Market Context & Opportunities

The pandemic created perfect conditions for book app:
- **Increased home time** drove reading behavior surge
- **Digital-first preferences** accelerated platform adoption
- **Community need** replaced physical book clubs with digital alternatives
- **Content discovery challenges** in overwhelming digital marketplace

Our dataset aligns perfectly with these trends, showing engaged users actively rating contemporary content.

## 4. Conclusions

The data reveals a clear path to success: build a community-driven book discovery platform that leverages authentic user engagement patterns rather than competing on pure algorithmic recommendations. By focusing on the engaged reader segment (demonstrated by our 6 power users) and quality contemporary content (81.9% post-2000 books), we can create a differentiated product that thrives in the post-COVID reading boom.

The market opportunity is validated by strong user engagement metrics, clear content preferences, and identifiable community leaders. Our strategy should prioritize community building over user acquisition, quality over quantity, and authentic engagement over algorithmic optimization.
